In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [138]:
cci = pd.read_csv("../data/CCI Price Index.csv", header=None)
cci = cci[cci.columns[[0,1,5,9,13,17]]].reset_index(drop=True)
cci = pd.concat([cci.iloc[[1, 3]], cci.iloc[8:2919]]).reset_index(drop=True)

h1 = cci.iloc[0].fillna('').astype(str).str.strip()
h2 = cci.iloc[1].fillna('').astype(str).str.strip()
new_cols = (h1 + ' ' + h2).str.replace(r'\s+', ' ', regex=True).tolist()
cci = cci.iloc[2:].copy()
cci.columns = new_cols
date_col = new_cols[0]
cci[date_col] = pd.to_datetime(cci[date_col])
cci = cci.set_index(date_col)
cci.index.name = 'Date'

In [139]:
cci.head(5)

,CCI 5500 FOB with VAT,CCI 5000 FOB with VAT,CCI 3800 Import CFR,CCI 4700 Import CFR,CCI Shanxi Low-sulphur Ex-plant with VAT
Date,,,,,
2025-07-04,621,553,395.12,511.14,1144
2025-07-03,620,552,395.12,511.14,1144
2025-07-02,620,552,395.67,511.86,1144
2025-07-01,620,551,393.5,509.52,1144
2025-06-30,620,550,393.5,509.52,1144


In [140]:
stocks = pd.read_csv("../data/Thermal Coal Stocks.csv", header=None)
stocks = stocks[stocks.columns[[0,2]]].reset_index(drop=True)
stocks = pd.concat([stocks.iloc[[1, 3]], stocks.iloc[6:2919]]).reset_index(drop=True)

h1 = stocks.iloc[0].fillna('').astype(str).str.strip()
h2 = stocks.iloc[1].fillna('').astype(str).str.strip()
new_cols = (h1 + ' ' + h2).str.replace(r'\s+', ' ', regex=True).tolist()

stocks = stocks.iloc[2:].copy()
stocks.columns = new_cols

date_col = new_cols[0]
stocks[date_col] = pd.to_datetime(stocks[date_col])
stocks = stocks.set_index(date_col)
stocks.index.name = 'Date'

In [141]:
stocks.head(5)

,Six Coastal Power Groups Days of Burn
Date,
2025-07-04,16.8
2025-07-03,16.9
2025-07-02,16.9
2025-07-01,16.8
2025-06-30,16.8


In [146]:
df = cci.join(stocks, how='inner')
df.sort_index(ascending=True, inplace=True)
df.head(5)

,CCI 5500 FOB with VAT,CCI 5000 FOB with VAT,CCI 3800 Import CFR,CCI 4700 Import CFR,CCI Shanxi Low-sulphur Ex-plant with VAT,Six Coastal Power Groups Days of Burn
Date,,,,,,
2016-10-08,585,531,334.23,465.72,NaN,23.4
2016-10-09,590,536,333.73,465.03,NaN,22.4
2016-10-10,592,537,341.51,474.97,NaN,23.5
2016-10-11,599,540,348.3,479.61,NaN,24.7
2016-10-12,602,544,351.18,488.19,NaN,23.5
